 ### Mô hình ngôn ngữ và ứng dụng cho kiểm lỗi chính tả

#### Mục tiêu: xây dựng chương trình kiểm lỗi cú pháp tiếng Anh đơn giản 

Bài 1:  

a)	Xây dựng mô hình ngôn ngữ dựa trên n-gram sử dụng phương pháp smoothing là Laplace, cho các mô hình:
-	1-gram
-	2-gram
-	3-gram

b)	Tính xác suất của một câu và tính Perplexity của một câu dựa theo 1-gram, 2-gram, 3-gram

c)	Phân tích kết quả



#### Đọc file input

In [1]:
# đọc file
filename='tedtalk.txt'
with open(filename, 'r', encoding='utf-8')  as file:
    lines=file.readlines()

max_lines = 10000 
lines=lines[:max_lines]
# drop \n:
lines=[line.strip() for line in lines]
print(len(lines))
print(lines[:5])     

10000
["Here are two reasons companies fail: they only do more of the same, or they only do what's new", 'To me the real, real solution to quality growth is figuring out the balance between two activities: exploration and exploitation', 'Both are necessary, but it can be too much of a good thing', 'Consider Facit', "I'm actually old enough to remember them"]


#### Cài đặt thư viện xử lý ngôn ngữ nltk

In [2]:
!pip install nltk


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to C:\Users\Gia
[nltk_data]     Bao\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

#### Thực hiện tokenize tập input
Thêm '\<s\>' vào trước mỗi câu, thêm '\</s\>' vào sau mỗi câu.

In [4]:
# tokenize sentences 
import nltk
sentences=[]
all_tokens_count=0

for line in lines:
    tokens=nltk.word_tokenize(line)
    tokens.insert(0,'<s>')
    tokens.append('</s>')
    all_tokens_count+=len(tokens)
    sentences.append(tokens)
    
print('all_tokens_count=',all_tokens_count)
print(len(sentences))
print(sentences[:5])

all_tokens_count= 206296
10000
[['<s>', 'Here', 'are', 'two', 'reasons', 'companies', 'fail', ':', 'they', 'only', 'do', 'more', 'of', 'the', 'same', ',', 'or', 'they', 'only', 'do', 'what', "'s", 'new', '</s>'], ['<s>', 'To', 'me', 'the', 'real', ',', 'real', 'solution', 'to', 'quality', 'growth', 'is', 'figuring', 'out', 'the', 'balance', 'between', 'two', 'activities', ':', 'exploration', 'and', 'exploitation', '</s>'], ['<s>', 'Both', 'are', 'necessary', ',', 'but', 'it', 'can', 'be', 'too', 'much', 'of', 'a', 'good', 'thing', '</s>'], ['<s>', 'Consider', 'Facit', '</s>'], ['<s>', 'I', "'m", 'actually', 'old', 'enough', 'to', 'remember', 'them', '</s>']]


#### Counting 1-gram

In [5]:
# counting 1-gram 
from collections import Counter
counter_unigram=Counter()

#ENTER YOU CODE HERE
#-> Dùng counter_unigram để đếm token 1-gram cho tập sentences
for sentence in sentences:
    counter_unigram.update(sentence)

print(counter_unigram.most_common(10))

V=len(counter_unigram)

print('V=',V)

n=0
for gram in counter_unigram:
    n+=counter_unigram[gram]
n=n-counter_unigram['<s>']-counter_unigram['</s>']


print('n=',n)
print(counter_unigram['the'])
print(counter_unigram['he'])

[(',', 11320), ('<s>', 10000), ('</s>', 10000), ('the', 7556), ('to', 4978), ('of', 4444), ('and', 4151), ('a', 3930), ('that', 3454), ('I', 3068)]
V= 13741
n= 186296
7556
289


#### Counting 2-gram

In [6]:
from nltk import ngrams
bi_grams=[]


#ENTER YOU CODE HERE
#-> Dùng ngrams để đếm token 2-gram cho tập sentences
for sentence in sentences:
    bi_grams.extend(ngrams(sentence,2))
#END YOUR CODE
    
print(len(bi_grams))

for i in range(3):
    print(bi_grams[i])

counter_bigram = Counter(bi_grams)

print('V=',len(counter_bigram))
print(counter_bigram[('here','are')])

196296
('<s>', 'Here')
('Here', 'are')
('are', 'two')
V= 81850
4


#### Counting 3-gram

In [7]:
tri_grams=[]

#ENTER YOU CODE HERE
#-> Dùng ngrams để đếm token 3-gram cho tập sentences
for sentence in sentences:
    tri_grams.extend(ngrams(sentence,3))
#END YOUR CODE
    
print(len(tri_grams))

for i in range(3):
    print(tri_grams[i])

counter_trigram = Counter(tri_grams)
print('V=',len(counter_trigram))
print(counter_trigram[('here','are','two')])

186296
('<s>', 'Here', 'are')
('Here', 'are', 'two')
('are', 'two', 'reasons')
V= 143772
1


#### Viết hàm tính xác suất cho từng loại: 1-gram, 2-gram, 3-gram

In [14]:
# tính prob theo từng loại: 1-gram, 2-gram, 3-gram
def uni_prob(word):
    #ENTER YOU CODE HERE
    #-> tính xác suất 1-gram
    return (counter_unigram[word]+1)/(n+V)
    #END YOUR CODE

def bi_prob(word1,word2):
    #ENTER YOU CODE HERE
    #-> tính xác suất 2-gram
    return (counter_bigram[(word1,word2)]+1)/(counter_unigram[word1]+V)
    #END YOUR CODE
    
def tri_prob(word1,word2,word3):
    #ENTER YOU CODE HERE
    #-> tính xác suất 3-gram
    return (counter_trigram[(word1,word2,word3)]+1)/(counter_bigram[(word1,word2)]+V)
    #END YOUR CODE


#### Viết hàm tính xác xuất cho một câu, normalize theo 1 từ

In [15]:
# tính xác suất của một câu, normalize theo 1 từ 
def probLM(sent,n):
    if n>3 or n<1: # không xét trường hợp này 
        return 0
    tokens=nltk.word_tokenize(sent.lower())
    tokens += ['<s>']+tokens
    prob=1
    for i in range(1,len(tokens)):
        if n==1:
            prob*=uni_prob(tokens[i])
        elif n==2:
            prob*=bi_prob(tokens[i-1],tokens[i])
        elif n==3:
            if i>=2:
                prob*=tri_prob(tokens[i-2],tokens[i-1],tokens[i])
            else:
                prob*=bi_prob(tokens[i-1],tokens[i])
    l=len(tokens)-1
    return prob**(1/l)

#### Kiểm tra xác xuất của 1 câu

In [19]:
# sent='the human body with new abilities is no longer a question'
# sent='the human body with new from abilities is no longer a question'
sent='A few years back from'
print('n=1')
pr=probLM(sent,1)
print('prob=',pr)
print('perplexity=',1/pr)

print('n=2')
pr=probLM(sent,2)
print('prob=',pr)
print('perplexity=',1/pr)

print('n=3')
pr=probLM(sent,3)
print('prob=',pr)
print('perplexity=',1/pr)

n=1
prob= 0.0024390094946983173
perplexity= 410.00250395650494
n=2
prob= 0.0004853876230056048
perplexity= 2060.2091042367865
n=3
prob= 0.00018897930287603375
perplexity= 5291.584765004546


#### So sánh xác suất của 2 câu

In [17]:
# kiểm tra xem 2 xâu có xác suất hơn nhau thế nào, ví dụ cho bài toán speech to text
sent1='the human body with new abilities is no longer a question'
sent2='the human body with knew abilities is know longer a question'

#ENTER YOU CODE HERE
#-> tính xác suất và perplexity cho 2 câu, hiển thị kết quả để sánh
pr1=probLM(sent1,3)
pr2=probLM(sent2,3)
print('prob1=',pr1)
print('prob2=',pr2)
perplexity1 = 1/pr1
perplexity2 = 1/pr2

print('perplexity1=',perplexity1)
print('perplexity2=',perplexity2)

#END YOUR CODE

prob1= 0.00018894862176008524
prob2= 0.00012480501222787032
perplexity1= 5292.44400242165
perplexity2= 8012.498714187771
